# Concha Tutorial
Follow this instructions to understand how to deal with Concha and its concepts:

* Concha Service
* Servers auxiliar mock Service
* Documents
* Tricks
* GET Tricks
* POST Tricks
* TREAT Tricks and dependent clauses

You should be executing this very notebook running `jupyter concha_tutorial.ipynb` in a console to follow the instructions to master Concha usage. Please run cells sequentially to do that.

In [38]:
# Please run first this cell to set up rest of executions
import requests

def post(uri, body):
    response = requests.post(url=uri, json=body)
    print('status={}'.format(response.status_code))
    print(response.text)

def get(uri):
    response = requests.get(url=uri)
    print('status={}'.format(response.status_code))
    print(response.text)

## Concha (v1) Service
Concha runs as a RESTful service you must execute in `concha/concha` directory running `python3 concha` in a console. It will offer an end-point in `localhost:5000`.

### Concha Resources
Concha deals with two resources: **documents** and **tricks**. Documents are the plain natural language sentences you tell Concha to interact with. Tricks are the "programs" you instruct Concha with to know how to deal with each document. Concha will select the right combination among all stored tricks to react to every document it receive.

In [39]:
# You should be able to verify concha service is up and running if execution returns an empty list:
"""
status=200
[]
"""
get('http://localhost:5000/v1/documents')

status=200
[]



## Servers (v1) auxiliar mock Service
In order to have some predictable mock service to interact with, you can execute in `concha/mocks` directory the command `python3 servers`. It will offer an end point in `localhost:6000`.

### Servers Resources
Servers offers one single resource: **servers**. It simulates the management of a fake cluster of servers (a micro cloud with no real servers behind) you can query, start and stop.

In [40]:
# You should be able to verify servers service is up and running if execution returns an empty JSON:
"""
status=200
{}
"""
get('http://localhost:6000/v1/servers')

status=200
{}



### Server Methods
There's just a plain CRUD set of methods:
* **POST /v1/servers/server_name** A new server named server_name (if it doesn't exist yet) with whatever JSON description in the body.
* **GET /v1/servers** The named set of all servers with its description and fake load.
* **GET /v1/servers/server_name** The description and fake load of server_name server.
* **PUT /v1/servers/server_name** Modify server named server_name (if it already exists) with whatever new JSON description in the body.
* **DEL /v1/servers/server_name** Delete server named server_name (if it already exists).

## Documents
Documents resources are stored in memory (no persistence yet) in a sequential way (append only).

### Documents Methods
Methods are as few as possible.
* **POST /v1/documents:analyzeSyntax** Fancy method to get the syntactic analysis of the given text. This is an instrumental function to help to think in the development of new _tricks_ (more on this later).

_Request body:_ The text you want Concha deal with
```javascript
{
	"text": "mi mamá me mima"
}
```

_Response body:_ A JSON structure with the parsing of the text according to the [Universal Dependencies annotation guidelines](http://universaldependencies.org). It's weird, but legible, and you can consider it as the internal Concha's assembler. You just need to know _ID_ is sequential order of the word in the text (starting by 1), _FORM_ is the word itself, and the _key_ of each compound object (i.e. _"nsubj"_ = nominal subject) is the [syntactic relationship](http://universaldependencies.org/u/dep/index.html) to the parent key.
```javascript
{
    "ROOT": {
        "FEATS": "Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin|fPOS=VERB++",
        "FORM": "mima",
        "ID": "4",
        "UPOSTAG": "VERB",
        "iobj": {
            "FEATS": "Case=Acc,Dat|Number=Sing|Person=1|PrepCase=Npr|PronType=Prs|Reflex=Yes|fPOS=PRON++",
            "FORM": "me",
            "ID": "3",
            "UPOSTAG": "PRON"
        },
        "nsubj": {
            "FEATS": "Gender=Fem|Number=Sing|fPOS=NOUN++",
            "FORM": "mamá",
            "ID": "2",
            "UPOSTAG": "NOUN",
            "det": {
                "FEATS": "Number=Sing|Person=1|Poss=Yes|PronType=Prs|fPOS=DET++",
                "FORM": "mi",
                "ID": "1",
                "UPOSTAG": "DET"
            }
        }
    }
}
```

In [41]:
# Let Concha do some syntactical analysis with your own phrases
post(
    'http://localhost:5000/v1/documents:analyzeSyntax',
    {"text": "mi mamá me mima"}
)

status=200
{
  "ROOT": {
    "FEATS": "Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin|fPOS=VERB++", 
    "FORM": "mima", 
    "ID": "4", 
    "UPOSTAG": "VERB", 
    "iobj": {
      "FEATS": "Case=Acc,Dat|Number=Sing|Person=1|PrepCase=Npr|PronType=Prs|Reflex=Yes|fPOS=PRON++", 
      "FORM": "me", 
      "ID": "3", 
      "UPOSTAG": "PRON"
    }, 
    "nsubj": {
      "FEATS": "Gender=Fem|Number=Sing|fPOS=NOUN++", 
      "FORM": "mam\u00e1", 
      "ID": "2", 
      "UPOSTAG": "NOUN", 
      "det": {
        "FEATS": "Number=Sing|Person=1|Poss=Yes|PronType=Prs|fPOS=DET++", 
        "FORM": "mi", 
        "ID": "1", 
        "UPOSTAG": "DET"
      }
    }
  }
}



* **POST /v1/documents** It is used as the way to _tell_ Concha new documents to deal with, and recover the details of the associated actions in the answer.

_Request body:_ The text you want Concha deal with
```javascript
{
	"text": "repite hola mundo"
}
```

_Response body:_ It holds the _answer_text_ (the very Concha's answer to the given document), a sequential _id_ numbering the documents as they arrive, the syntactical analysis of the document _request_ (equal to the documents:analyzeSyntax response) and a list of the applied _tricks_ in the order they where used.
```javascript
{
    "answer_text": "hola mundo",
    "id": 3,
    "request": {
        "ROOT": {
            "FEATS": "Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++",
            "FORM": "repite",
            "ID": "1",
            "UPOSTAG": "VERB",
            "dobj": {
                "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++",
                "FORM": "mundo",
                "ID": "3",
                "UPOSTAG": "NOUN",
                "amod": {
                    "FEATS": "Number=Sing|fPOS=ADJ++",
                    "FORM": "hola",
                    "ID": "2",
                    "UPOSTAG": "ADJ"
                }
            }
        }
    },
    "tricks": [
        0
    ]
}```

It has not been explained yet a clue about how to deal with _tricks_, so any usage up to now should fail returning a Concha proprietary error status code (600).

In [42]:
# There should yield a failure sending a document when Concha hasn learnt no trick yet:
"""
status=600
{
  "answer_text": "{'FORM': 'NoTrick'}", 
  "id": ......
"""
post(
    'http://localhost:5000/v1/documents',
    {"text": "repite hola mundo"}
)

status=600
{
  "answer_text": "{'FORM': 'NoTrick'}", 
  "id": 0, 
  "request": {
    "ROOT": {
      "FEATS": "Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++", 
      "FORM": "repite", 
      "ID": "1", 
      "UPOSTAG": "VERB", 
      "dobj": {
        "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++", 
        "FORM": "mundo", 
        "ID": "3", 
        "UPOSTAG": "NOUN", 
        "amod": {
          "FEATS": "Number=Sing|fPOS=ADJ++", 
          "FORM": "hola", 
          "ID": "2", 
          "UPOSTAG": "ADJ"
        }
      }
    }
  }, 
  "tricks": []
}



* **GET /v1/documents** It returns the list of previously received and processed documents to date, with an associated timestamp.

_Response body:_ A list with the _date_ timestamp and the _text_ of each of the previously received documents.
```javascript
[
    {
        "date": "2018-02-06 21:04:15",
        "text": "repite hola mundo"
    },
    {
        "date": "2018-02-06 23:09:21",
        "text": "repite lo más difícil que te salga"
    }
]
```

In [43]:
# Documents list should hold all the (failed) documents you have sent up to now:
get('http://localhost:5000/v1/documents')

status=200
[
  {
    "date": "2018-02-08 00:55:06", 
    "text": "repite hola mundo"
  }
]



## Tricks
Tricks resources are stored in memory (no persistence yet) in a sequential way (append only).

### Trick structure
Every trick has an _antecedent_ (`given`) and a _consequent_ (`then`) part, and some (most of them) have an optional _action_ (`when`) part. Any trick will be triggeed if the current document (or its derivatives) matches the `given`. In that case, if there's a `when` part it will be executed. Lastly, the `then` part will generate a final or intermediate answer with the combination of some new words, part of the incoming document and/or part of the action response. Tricks will be recursively self-assembled to get to the appropriate answer.
```javascript
{
	"given": {
		"ROOT": {
			"FORM": "da",
			"dobj": {
				"FORM": "*algo"
			},
			"iobj": {
				"FORM": "*alguien",
				"case": {
					"FORM": "a"
				}
			}
		}
	},
	"then": {
		"200": "Le voy a dar {d[ROOT][dobj]} , si no hay inconveniente , {d[ROOT][iobj]}",
		"400": "No puedo darlo, no me has especificado qué dar o a quién"
	}
}
```
**Given part** holds a subset of a Universal Dependencies syntactic analysis, holding the most important parts of your favourite phrase you want to react to. Usually you will focus on _FORM_ fields (the very words) and the syntactical dependencies (keys of the object tree). The values of the _FORM_ fields have a different meaning according to its first letter:
* **No special starting character:** The word must match as is.
* **Word starting by ~ character:** It means it could match with a similar word (not implemented yet, doing right now an "as is" matching).
* **Word starting by * character:** It means it will match with any word (remaining word acts as a explainatory comment).

**When part** will be explained later as it is used. Tricks with no `when` part are actionless tricks who act just as word mappers.

** Then part** will generate a combined text response according to the status code of the action, if any (actionless tricks are assumed to always return `200`).

### Trick sub-text selection format
In order to get a part of a text, Concha uses the `python str.format()` syntax, nesting `object[child][grandchild]...`. Those references will be replaced for the full branch the references points to (i.e. the full direct object, as much words it is built of). When accessing part of the incoming _document_ (to the trick, it could be an intermediate incoming document), references will start by letter "d", as you can see in the previous example. When accesing the action _response_ they will start by letter "r", followed by `[body]` (it would access in a future to header content).

### Tricks Methods
Methods are as few as possible.
* **POST /v1/tricks** Sets up a new trick.

_Request body:_ The trick as stated in the trick structure explanation.

_Response body:_ A JSON indicating the sequential _id_ of the new trick and a message with the final status (HTTP status code indicates if there's some issue). It uses to return a 201 code.

In [44]:
# Let's teach Concha its first new trick:
post(
    'http://localhost:5000/v1/tricks',
    {
	"given": {
		"ROOT": {
			"FORM": "repite",
			"dobj": {
				"FORM": "*algo"
			}
		}
	},
	"then": {
		"200": "{d[ROOT][dobj]}",
		"400": "No puedo repetirlo, no me has especificado qué repetir"
	}
}
)

status=201
{
  "id": 0, 
  "message": "trick 0 created Ok"
}



In [45]:
# Let make Concha do the first "hello world"
post(
    'http://localhost:5000/v1/documents',
    {"text": "repite hola mundo"}
)

status=201
{
  "answer_text": "hola mundo", 
  "id": 1, 
  "request": {
    "ROOT": {
      "FEATS": "Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|fPOS=VERB++", 
      "FORM": "repite", 
      "ID": "1", 
      "UPOSTAG": "VERB", 
      "dobj": {
        "FEATS": "Gender=Masc|Number=Sing|fPOS=NOUN++", 
        "FORM": "mundo", 
        "ID": "3", 
        "UPOSTAG": "NOUN", 
        "amod": {
          "FEATS": "Number=Sing|fPOS=ADJ++", 
          "FORM": "hola", 
          "ID": "2", 
          "UPOSTAG": "ADJ"
        }
      }
    }
  }, 
  "tricks": [
    0
  ]
}

